# Initiliation

In [1]:
!pip install transformers datasets --quiet

     |████████████████████████████████| 3.1 MB 11.3 MB/s 
     |████████████████████████████████| 298 kB 6.6 MB/s 
     |████████████████████████████████| 61 kB 279 kB/s 
     |████████████████████████████████| 3.3 MB 42.3 MB/s 
     |████████████████████████████████| 596 kB 46.2 MB/s 
     |████████████████████████████████| 895 kB 46.3 MB/s 
     |████████████████████████████████| 243 kB 51.2 MB/s 
     |████████████████████████████████| 132 kB 25.9 MB/s 
     |████████████████████████████████| 1.1 MB 48.4 MB/s 
     |████████████████████████████████| 192 kB 42.7 MB/s 
     |████████████████████████████████| 271 kB 49.1 MB/s 
     |████████████████████████████████| 160 kB 44.8 MB/s 


In [2]:
from transformers import TrainingArguments
from transformers import Trainer
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer

from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, classification_report, confusion_matrix
from datasets import Dataset
from datasets import load_metric

import numpy as np
import math
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from google.colab import drive

# Data Preparation

In [3]:
drive.mount('/content/drive/')

Mounted at /content/drive/


In [4]:
%cd 'drive/MyDrive/Masterarbeit/Colab Notebooks/OVERVIEW MBTI/Datasets/URL_Balanced_MASK'

/content/drive/.shortcut-targets-by-id/1aHXlqhpj1STohhfU4gn53D4whaLH__Jz/Masterarbeit/Colab Notebooks/OVERVIEW MBTI/Datasets/URL_Balanced_MASK


In [5]:
dfTF = pd.read_csv('MBTI_TF_URL_MASK.csv', sep=",", error_bad_lines=False)
dfTF

,text,label
0,i like that you are kind as [MASK] i find that...,1
1,oh my you are right who really talks like tha...,1
2,yep yep yep especially the last one yep agree ...,1
3,things that are generalizable to the entire po...,1
4,work student hobbies studying gaming reading d...,0
...,...,...
6935,well mostly i dont like avocado but the primar...,1
6936,during an argument rather than trying to valid...,1
6937,cigarettes are like hamsters perfectly harmles...,1
6938,bookshelf porn as a nonamerican please excuse ...,0


# Model Training

In [6]:
modeltype = "princeton-nlp/sup-simcse-bert-base-uncased"

In [7]:
train, test = train_test_split(dfTF, test_size=0.2, random_state=0, stratify=dfTF.label)

train = Dataset.from_pandas(train)
test = Dataset.from_pandas(test)

tokenizer = AutoTokenizer.from_pretrained(modeltype)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_train = train.map(tokenize_function, batched=True)
tokenized_test = test.map(tokenize_function, batched=True)

full_train_dataset = tokenized_train
full_eval_dataset = tokenized_test

model = AutoModelForSequenceClassification.from_pretrained(modeltype, num_labels=2)

training_args = TrainingArguments(
    "SIMCSE_BERT_TF_MASK", 
    evaluation_strategy="epoch",
    save_strategy = 'no',
    save_steps = 100000,
    save_total_limit = 1,
    metric_for_best_model="eval_f1")

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    print(classification_report(labels, preds, labels=[0,1]))
    print(confusion_matrix(labels,preds))
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

Downloading:   0%|          | 0.00/252 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/689 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at princeton-nlp/sup-simcse-bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Hyperparameter Optimization

In [8]:
! pip install optuna --quiet

     |████████████████████████████████| 308 kB 10.8 MB/s 
     |████████████████████████████████| 209 kB 48.3 MB/s 
     |████████████████████████████████| 80 kB 8.3 MB/s 
     |████████████████████████████████| 75 kB 4.4 MB/s 
     |████████████████████████████████| 49 kB 5.5 MB/s 
     |████████████████████████████████| 149 kB 49.4 MB/s 
     |████████████████████████████████| 112 kB 47.5 MB/s 


In [9]:
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(modeltype, num_labels=2)

In [10]:
trainer = Trainer(
      model_init=model_init,
      args=training_args, 
      train_dataset=full_train_dataset, 
      eval_dataset=full_eval_dataset,
      compute_metrics=compute_metrics 
  )

loading configuration file https://huggingface.co/princeton-nlp/sup-simcse-bert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/886dba277a27c6ab50ab3d0bfd8839d354cfeed717289623026415c62b687338.1b14bcddba43d86a607eedb4b638b87d30aa00c839358953dbd36f2cd3317c83
Model config BertConfig {
  "_name_or_path": "result/bert-base-uncased-cls_before_pooler-sym_mlp-mlp_bert-bs64-gpu8-gs1-lr5e-5-m=stsb-norm0.05-l32-contra",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.12.5",
  "type_vocab_size": 2,
 

In [11]:
import sklearn.metrics as metrics
import optuna
import sys
import logging

def objective (metrics):
  return metrics['eval_f1']

def hyperparameter_space(trial):

    return {
        "learning_rate": trial.suggest_float("learning_rate", 5e-8, 5e-1, log=True),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [2, 4, 8, 16]),
        "weight_decay": trial.suggest_float("weight_decay", 5e-12, 5e-1, log=True),
        "num_train_epochs": trial.suggest_float("num_train_epochs",1,8,log=True),
        #"adam_epsilon": trial.suggest_float("adam_epsilon", 1e-10, 1e-6, log=True),
        #"seed" : trial.suggest_float("seed",10,60,log=True)
        }

optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))
study_name = "SIMCSE_BERT_MASK_TF"  # Unique identifier of the study.
storage_name = "sqlite:///{}.db".format(study_name)

best_run = trainer.hyperparameter_search(hp_space=hyperparameter_space,compute_objective=objective, n_trials=50, direction="maximize",study_name=study_name, storage=storage_name )

study = optuna.create_study()

[I 2021-12-07 21:27:13,913] A new study created in RDB with name: SIMCSE_BERT_MASK_TF


A new study created in RDB with name: SIMCSE_BERT_MASK_TF


Trial:
loading configuration file https://huggingface.co/princeton-nlp/sup-simcse-bert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/886dba277a27c6ab50ab3d0bfd8839d354cfeed717289623026415c62b687338.1b14bcddba43d86a607eedb4b638b87d30aa00c839358953dbd36f2cd3317c83
Model config BertConfig {
  "_name_or_path": "result/bert-base-uncased-cls_before_pooler-sym_mlp-mlp_bert-bs64-gpu8-gs1-lr5e-5-m=stsb-norm0.05-l32-contra",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.12.5",
  "type_vocab_size

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.677400,0.662608,0.623919,0.577061,0.647011,0.598964
2,0.657000,0.625932,0.663545,0.637052,0.680015,0.644552
3,0.605400,0.595617,0.698847,0.689882,0.700235,0.689158
4,0.583300,0.582110,0.698847,0.686449,0.704520,0.686562
5,0.578400,0.570028,0.708934,0.701433,0.709630,0.700363
6,0.561300,0.566542,0.708213,0.699901,0.709775,0.698923
6,0.554800,0.565216,0.709654,0.701530,0.711131,0.700506


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, __index_level_0__.
***** Running Evaluation *****
  Num examples = 1388
  Batch size = 8


...

              precision    recall  f1-score   support

           0       0.46      1.00      0.63       632
           1       0.00      0.00      0.00       756

    accuracy                           0.46      1388
   macro avg       0.23      0.50      0.31      1388
weighted avg       0.21      0.46      0.28      1388

[[632   0]
 [756   0]]


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` param

Trial 49 pruned. 


[I 2021-12-08 07:41:42,450] A new study created in memory with name: no-name-ce91c0fc-4eae-400e-9f62-3be4727526a8


A new study created in memory with name: no-name-ce91c0fc-4eae-400e-9f62-3be4727526a8


In [12]:
storage_name

'sqlite:///SIMCSE_BERT_MASK_TF.db'

In [13]:
study_name

'SIMCSE_BERT_MASK_TF'

In [14]:
study = optuna.create_study(study_name=study_name, storage=storage_name, load_if_exists=True, direction="maximize")
df = study.trials_dataframe(attrs=("number", "value", "params", "state"))

[I 2021-12-08 07:41:42,572] Using an existing study with name 'SIMCSE_BERT_MASK_TF' instead of creating a new one.


Using an existing study with name 'SIMCSE_BERT_MASK_TF' instead of creating a new one.


In [15]:
df

,number,value,params_learning_rate,params_num_train_epochs,params_per_device_train_batch_size,params_weight_decay,state
0,0,0.701530,5.113165e-07,6.905193,8,4.337244e-04,COMPLETE
1,1,0.727609,2.895712e-05,4.033449,8,1.110801e-09,COMPLETE
2,2,0.752182,1.038324e-04,2.169266,16,4.717749e-07,COMPLETE
3,3,0.312871,6.032013e-02,3.150463,4,5.597112e-09,COMPLETE
4,4,0.352612,3.416136e-04,5.336966,8,1.925918e-02,COMPLETE
5,5,0.499189,3.405867e-07,1.906998,4,1.597199e-09,PRUNED
6,6,0.312871,1.309907e-01,2.800622,4,1.297187e-04,PRUNED
7,7,0.312871,1.109630e-02,3.710493,8,3.778855e-08,PRUNED
8,8,0.476520,9.091989e-08,4.166444,16,5.388756e-07,PRUNED
9,9,0.739129,3.515249e-06,7.852373,2,1.586525e-04,COMPLETE


In [16]:
fig = optuna.visualization.plot_param_importances(study)
fig.show()

In [17]:
best_run

BestRun(run_id='20', objective=0.7613559108152659, hyperparameters={'learning_rate': 9.550187523162385e-06, 'num_train_epochs': 2.391488658266402, 'per_device_train_batch_size': 2, 'weight_decay': 7.344308135535765e-11})

In [18]:
optuna.visualization.plot_intermediate_values(study)

In [19]:
optuna.visualization.plot_parallel_coordinate(study)

In [20]:
optuna.visualization.plot_optimization_history(study)

In [21]:
optuna.visualization.plot_contour(study)

In [22]:
optuna.visualization.plot_slice(study)

In [23]:
optuna.visualization.plot_edf(study)